In [1]:
import Pkg; Pkg.activate(@__DIR__)

  Activating project at `~/Documents/eth_courses/notebooks/control/attitude`


In [738]:
using Rotations
using LinearAlgebra
using GLMakie
using ModelingToolkit

GLMakie.activate!(inline=false)

## Derivation for Splitting quaternion error into reduced error and yaw error

In [791]:
let

@variables q0, q1, q2, q3
    
q_e = QuatRotation(q0, q1, q2, q3, false)

q_e_yaw = QuatRotation(q0, 0, 0, q3, false)

q_e_red = q_e * inv(q_e_yaw)

@show q_e_red.q
    
end;

q_e_red.q = Quaternions.Quaternion{Num}(q0^2 + q3^2, q0*q1 - q2*q3, q0*q2 + q1*q3, 0)


## Brescianini function

In [762]:
# reduced attitude error
function get_reduced_att_error(q_e)

    q0 = q_e.q.s
    q1 = q_e.q.v1
    q2 = q_e.q.v2
    q3 = q_e.q.v3

    denom = 1 / sqrt(q0^2 + q3^2)

    # reduced attitude error
    v1 = q0^2 + q3^2
    v2 = q0*q1 - q2*q3
    v3 = q0*q2 + q1*q3
    v4 = 0
    
    q_e_reduced = denom * [v1; v2; v3; v4]

    # yaw attitude error
    v1 = q0
    v2 = 0
    v3 = 0
    v4 = q3

    q_e_yaw= denom * [v1; v2; v3; v4]
    
    return QuatRotation(q_e_reduced), QuatRotation(q_e_yaw)
end

get_reduced_att_error (generic function with 1 method)

In [824]:
# current attitude
q = QuatRotation(RotZYX(roll=-0.2, pitch=-0.7, yaw=-0.2))

# desired attitude
q_d = QuatRotation(RotZYX(roll=0.3, pitch=0.1, yaw=0.2))

# error quaternion
# q_e =  q_d * inv(q)
q_e =  inv(q)* q_d

(q_e_reduced_bres, q_e_yaw_bres) =  get_reduced_att_error(q_e)

@show q_e_reduced_bres.q
@show q_e_yaw_bres.q

q_e_reduced_bres.q = Quaternions.QuaternionF64(0.8768461720163, 0.19891353885962396, 0.4376918946915765, 0.0)
q_e_yaw_bres.q = Quaternions.QuaternionF64(0.9809166624997849, 0.0, 0.0, 0.19442865331602538)


Quaternions.QuaternionF64(0.9809166624997849, 0.0, 0.0, 0.19442865331602538)

## Verification

In [825]:
# res_bres =   q_e_yaw_bres * q_e_reduced_bres
res_bres =   q_e_reduced_bres * q_e_yaw_bres 

@show q_e.q
@show res_bres.q

q_e.q = Quaternions.QuaternionF64(0.8601130205799412, 0.28021745031642636, 0.390664781057267, 0.17048402039044114)
res_bres.q = Quaternions.QuaternionF64(0.8601130205799413, 0.2802174503164264, 0.390664781057267, 0.17048402039044114)


Quaternions.QuaternionF64(0.8601130205799413, 0.2802174503164264, 0.390664781057267, 0.17048402039044114)

In [794]:
res = q_e * q

res_bres =   q_e_reduced_bres * q_e_yaw_bres * q

@show res.q
@show res_bres.q

res.q = Quaternions.QuaternionF64(0.9833474432563559, 0.14357217502739192, 0.06407134770607124, 0.09115754934299078)
res_bres.q = Quaternions.QuaternionF64(0.9833474432563559, 0.14357217502739195, 0.06407134770607126, 0.0911575493429908)


Quaternions.QuaternionF64(0.9833474432563559, 0.14357217502739195, 0.06407134770607126, 0.0911575493429908)

In [795]:
Rotations.params(RotZYX(q_e_reduced_mine * q))

3-element StaticArraysCore.SVector{3, Float64} with indices SOneTo(3):
 -0.2535415177195018
  0.20476851597992113
 -0.004102977145876809

# Plotting

In [796]:
to_makie_quaternion(q::QuatRotation) =  Quaternion(q.q.v1, q.q.v2, q.q.v3, q.q.s)

plot_reset(q_init) = GLMakie.rotate!(quad_model, to_makie_quaternion(q_init))

plot_reset (generic function with 2 methods)

## Setup plot

In [768]:
using FileIO
using GLMakie


quad_stl = load(assetpath(pwd() * "/assets/cf2_assembly.obj"))

fig = Figure()

ax_lim = 0.05
ax = Axis3(fig[1, 1], aspect = :equal, title = "aspect = :equal", limits=(-ax_lim, ax_lim,-ax_lim, ax_lim,-ax_lim, ax_lim))

# static transparent model to indicate desired attitude
quad_model_des_att = mesh!(ax, quad_stl, color = RGBAf(1, 0, 0, 0.25),transparency=true)
GLMakie.rotate!(quad_model_des_att, to_makie_quaternion(q_d))

# model of movable quad
quad_model = mesh!(ax, quad_stl, color =:green)

display(fig)

GLMakie.Screen(...)

## Full rotation

In [797]:
let

plot_reset()  
sleep(1)
    
GLMakie.rotate!(quad_model, to_makie_quaternion(res))
end

0.9833474 + 0.14357218im + 0.06407135jm + 0.09115755km

## Reduced error + yaw rotation (brescianini)

In [831]:
let

plot_reset(q)  
sleep(1)
    
GLMakie.rotate!(quad_model, to_makie_quaternion( q * q_e_reduced_bres ))
sleep(1)
    
GLMakie.rotate!(quad_model, to_makie_quaternion(q * q_e_reduced_bres *q_e_yaw_bres  ))
sleep(1)

end